In [6]:
import os
import openai
from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType

openai.api_type = "azure"
openai.api_base = "https://cog-oaxatpknic6wq.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = "a4afc38d3a6246d3b855a8adcca0d0f1"

endpoint = "https://gptkb-oaxatpknic6wq.search.windows.net"
index_name = "gptkbindex-oaxatpknic6wq"

model_name = "gpt-35-turbo"
deployment_name ="chat"
prompt = "what is the capital of Germany?"

KB_FIELDS_CONTENT = os.environ.get("KB_FIELDS_CONTENT") or "content"
KB_FIELDS_CATEGORY = os.environ.get("KB_FIELDS_CATEGORY") or "category"
KB_FIELDS_SOURCEPAGE = os.environ.get("KB_FIELDS_SOURCEPAGE") or "sourcepage"


azure_credential = DefaultAzureCredential()

search_client = SearchClient(endpoint, index_name, credential=azure_credential) 

In [7]:
# ChatGPT uses a particular set of tokens to indicate turns in conversations
prompt_prefix = """<|im_start|>system
Assistant helps the company employees with their healthcare plan questions and employee handbook questions. 
Answer ONLY with the facts listed in the list of sources below. If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question. 
Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brackets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].

Sources:
{sources}

<|im_end|>"""

turn_prefix = """
<|im_start|>user
"""

turn_suffix = """
<|im_end|>
<|im_start|>assistant
"""

prompt_history = turn_prefix

history = []

summary_prompt_template = """Below is a summary of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base. Generate a search query based on the conversation and the new question. Source names are not good search terms to include in the search query.

Summary:
{summary}

Question:
{question}

Search query:
"""

In [9]:
# Execute this cell multiple times updating user_input to accumulate chat history
user_input = "What is the capital of Germany?"

# Exclude category, to simulate scenarios where there's a set of docs you can't see
exclude_category = None

if len(history) > 0:
    completion = openai.Completion.create(
        prompt = prompt ,
        max_tokens = 10,
        temperature = 0.5,
        engine=deployment_name)
    search = completion.choices[0].text
else:
    search = user_input

# Alternatively simply use search_client.search(q, top=3) if not using semantic search
print("Searching:", search)
print("-------------------")
filter = "category ne '{}'".format(exclude_category.replace("'", "''")) if exclude_category else None
r = search_client.search(search, 
                         filter=filter,
                         query_type=QueryType.SEMANTIC, 
                         query_language="en-us", 
                         query_speller="lexicon", 
                         semantic_configuration_name="default", 
                         top=3)
print (r)
results = [doc[KB_FIELDS_SOURCEPAGE] + ": " + doc[KB_FIELDS_CONTENT].replace("\n", "").replace("\r", "") for doc in r]
content = "\n".join(results)

prompt = prompt_prefix.format(sources=content) + prompt_history + user_input + turn_suffix

completion = openai.Completion.create(
    prompt = prompt ,
    max_tokens = 1024,
    temperature = 0.7,
    engine=deployment_name,
    stop=["<|im_end|>", "<|im_start|>"])

prompt_history += user_input + turn_suffix + completion.choices[0].text + "\n<|im_end|>" + turn_prefix
history.append("user: " + user_input)
history.append("assistant: " + completion.choices[0].text)

print("\n-------------------\n".join(history))
print("\n-------------------\nPrompt:\n" + prompt)

Searching: What is the capital of Germany?
-------------------
<iterator object azure.core.paging.ItemPaged at 0x1c14df20370>


ClientAuthenticationError: Operation returned an invalid status 'Unauthorized'

In [44]:
print(completion)

{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\",\"Berlin\", \"Hamburg\", \"Mun"
    }
  ],
  "created": 1686153874,
  "id": "cmpl-7OpwACoPNYlxy8gV1Ak4CUXkCi5Ja",
  "model": "gpt-35-turbo",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 7,
    "total_tokens": 17
  }
}
